In [1]:
%load_ext autoreload

In [2]:
%autoreload 2

In [3]:
import importlib

In [4]:
import vsms
import torch
import torch.nn as nn

In [5]:
import clip

In [173]:
from vsms import *

In [174]:
from vsms import BoxFeedbackQuery

In [39]:
class StringEncoder(object):
    def __init__(self):
        variant ="ViT-B/32"
        device='cpu'
        jit = False
        self.device = device
        model, preproc = clip.load(variant, device=device,  jit=jit)
        self.model = model
        self.preproc = preproc
        self.celoss = nn.CrossEntropyLoss(reduction='none')
        
    def encode_string(self, string):
        model = self.model.eval()
        with torch.no_grad():
            ttext = clip.tokenize([string])
            text_features = model.encode_text(ttext.to(self.device))
            text_features = text_features / text_features.norm(dim=-1, keepdim=True)
            return text_features.detach().cpu().numpy()

def get_text_features(self, actual_strings, target_string):        
    s2id = {}
    sids = []
    s2id[target_string] = 0
    for s in actual_strings:
        if s not in s2id:
            s2id[s] = len(s2id)

        sids.append(s2id[s])

    strings = [target_string] + actual_strings
    ustrings = list(s2id)
    stringids = torch.tensor([s2id[s] for s in actual_strings], dtype=torch.long).to(self.device)
    tstrings = clip.tokenize(ustrings)
    text_features = self.model.encode_text(tstrings.to(self.device))
    text_features = text_features / text_features.norm(dim=-1, keepdim=True)
    return text_features, stringids, ustrings
        
def forward(self, imagevecs, actual_strings, target_string):
    ## uniquify strings    
    text_features, stringids, ustrings = get_text_features(self, actual_strings, target_string)

    image_features = torch.from_numpy(imagevecs).type(text_features.dtype)
    image_features = image_features / image_features.norm(dim=-1, keepdim=True)
    image_features = image_features.to(self.device)
    scores = image_features @ text_features.t()
    
    assert scores.shape[0] == stringids.shape[0]
    return scores, stringids.to(self.device), ustrings

def forward2(self, imagevecs, actual_strings, target_string):
    text_features, stringids, ustrings = get_text_features(self, actual_strings, target_string)
    actual_vecs = text_features[stringids]
    sought_vec = text_features[0].reshape(1,-1)
    
    image_features = torch.from_numpy(imagevecs).type(text_features.dtype)
    image_features = image_features / image_features.norm(dim=-1, keepdim=True)
    image_features = image_features.to(self.device)

    search_score = image_features @ sought_vec.reshape(-1)
    confounder_score = (image_features * actual_vecs).sum(dim=1)
    return search_score, confounder_score
    
import torch.optim


In [8]:
import torch.nn.functional as F

In [9]:
nn.HingeEmbeddingLoss()

HingeEmbeddingLoss()

In [11]:
import ray
ray.init('auto')
xclip = ModelService(ray.get_actor('clip'))

2021-06-28 20:33:25,629	INFO worker.py:640 -- Connecting to existing Ray cluster at address: 192.168.1.18:6379


In [12]:
from vsms import *
benchparams = dict(
    objectnet=dict(loader=objectnet_cropped, idxs=np.load('./data/object_random_idx.npy')[:10000])
)

def load_ds(evs, dsnames):
    for k,v in tqdm(benchparams.items(), total=len(benchparams)):
        if k in dsnames:
            def closure():
                ev0 = v['loader'](xclip)
                idxs = v['idxs']
                idxs = np.sort(idxs) if idxs is not None else None
                ev = extract_subset(ev0, idxsample=idxs)
                evs[k] = ev
            closure()

In [13]:
evs = {}
load_ds(evs, 'objectnet')

  0%|          | 0/1 [00:00<?, ?it/s]

In [14]:
ev = evs['objectnet']

In [180]:
vecs = ev.embedded_dataset
hdb = AugmentedDB(raw_dataset=ev.image_dataset, embedding=ev.embedding,
                  embedded_dataset=vecs, vector_meta=ev.fine_grained_meta)

In [166]:
def show_scores(se, vecs, actual_strings, target_string):
    with torch.no_grad():
        se.model.eval()
        scs,stids,rawstrs = forward(se, vecs, actual_strings, target_string=target_string)
    scdf = pd.DataFrame({st:col  for  st,col in zip(rawstrs,scs.cpu().numpy().transpose())})
    display(scdf.style.highlight_max(axis=1))

def get_feedback(idxbatch):
    strids = np.where(ev.query_ground_truth.iloc[idxbatch])[1]
    strs = ev.query_ground_truth.columns[strids]
    strs = [search_terms['objectnet'][fbstr] for fbstr in strs.values]
    return strs

In [167]:
curr_firsts = pd.read_parquet('./data/cats_objectnet_ordered.parquet')

In [175]:
class Updater(object):
    def __init__(self, se, lr, rounds=1, losstype='hinge'):
        self.se = se
        self.losstype=losstype
        self.opt = torch.optim.AdamW([{'params': se.model.ln_final.parameters()},
                          {'params':se.model.text_projection},
#                          {'params':se.model.transformer.parameters(), 'lr':lr*.01}
                                     ], lr=lr, weight_decay=0.)
#        self.opt = torch.optim.Adam@([{'params': se.model.parameters()}], lr=lr)
        self.rounds = rounds
        
    def update(self, imagevecs, actual_strings, target_string):
        se = self.se
        se.model.train()
        losstype = self.losstype
        opt = self.opt
        margin = .3

        def opt_closure():
            opt.zero_grad()            
            if losstype=='ce':
                scores, stringids, rawstrs = forward(se, imagevecs, actual_strings, target_string)
                # breakpoint()
                iidx = torch.arange(scores.shape[0]).long()
                actuals = scores[iidx, stringids]
                midx = scores.argmax(dim=1)
                maxes = scores[iidx, midx]                
            elif losstype=='hinge':
                #a,b = forward2(se, imagevecs, actual_strings, target_string)
                scores, stringids, rawstrs = forward(se, imagevecs, actual_strings, target_string)
                # breakpoint()
                iidx = torch.arange(scores.shape[0]).long()
                maxidx = scores.argmax(dim=1)
                
                actual_score = scores[iidx, stringids].reshape(-1,1)
                #max_score = scores[iidx, maxidx]
                
                
                #target_score = scores[:,0]
                losses1 = F.relu(- (actual_score - scores - margin))
                #losses2 = F.relu(- (actual_score - target_score - margin))
                #losses = torch.cat([losses1, losses2])
                losses = losses1
            else:
                assert False
            loss = losses.mean()
            #print(loss.detach().cpu())
            loss.backward()

        for _ in range(self.rounds):
            opt.step(opt_closure)



def closure(search_query, max_n, firsts, show_display=False, batch_size=10):
    sq = search_terms['objectnet'][search_query]
    se = StringEncoder()
    up = Updater(se, lr=.0001, rounds=1)
    bs = batch_size
    bfq = BoxFeedbackQuery(hdb, batch_size=bs, auto_fill_df=None)
    tvecs = []
    dbidxs = []
    accstrs = []
    gts = []
    while True:
        tvec = se.encode_string(sq)
        tvecs.append(tvec)
        idxbatch, _ = bfq.query_stateful(mode='dot', vector=tvec, batch_size=bs)
        dbidxs.append(idxbatch)
        gtvals = ev.query_ground_truth[search_query][idxbatch].values
        gts.append(gtvals)
        if show_display:
            display(hdb.raw.show_images(idxbatch))
            display(gtvals)
        #vecs = ev.embedded_dataset[idxbatch]
        actual_strings = get_feedback(idxbatch)
        accstrs.extend(actual_strings)

        if show_display:
            display(actual_strings)
        if gtvals.sum() > 0 or len(accstrs) > max_n:
            break

    #     vcs = ev.embedded_dataset[idxbatch]
    #     astrs = actual_strings    
        vcs = ev.embedded_dataset[np.concatenate(dbidxs)]
        astrs = accstrs

        if show_display:
            show_scores(se, vcs, astrs, target_string=sq)
            
        up.update(vcs, actual_strings=astrs, target_string=sq)

        if show_display:
            show_scores(se, vcs, astrs, target_string=sq)


    frsts = np.where(np.concatenate(gts).reshape(-1))[0]
    if frsts.shape[0] == 0:
        firsts[search_query] = np.inf
    else:
        firsts[search_query] = frsts[0] + 1

In [176]:
cf = curr_firsts[curr_firsts.nfirst_x > batch_size]

In [182]:
x.category

'eraser (white board)'

In [181]:
firsts = {}
batch_size = 10
for x in tqdm(curr_firsts.itertuples()):
    closure(x.category, max_n=30, firsts=firsts, show_display=True, batch_size=batch_size)
    print(firsts[x.category], x.nfirst_x)
    if x.nfirst_x <= batch_size:
        break

0it [00:00, ?it/s]

MImageGallery(image_urls=['http://clauslocal:8000//nvme_drive/orm/nbs/vloop_notebooks/data/objectnet/images/sp…

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

['sponge',
 'butter',
 'pencil',
 'ice',
 'paper towel',
 'thermometer',
 'floss container',
 'shampoo bottle',
 'paper towel',
 'marker']

,white board eraser,sponge,butter,pencil,ice,paper towel,thermometer,floss container,shampoo bottle,marker
0,0.330947,0.269882,0.250114,0.249234,0.238518,0.258222,0.224474,0.284546,0.258418,0.222203
1,0.321016,0.219361,0.239310,0.265917,0.241428,0.239907,0.249936,0.295555,0.257949,0.253100
2,0.320297,0.221463,0.198804,0.277846,0.241380,0.225563,0.260534,0.254728,0.220268,0.249575
3,0.319571,0.238505,0.239196,0.244080,0.279864,0.261743,0.230328,0.291687,0.260539,0.230258
4,0.316564,0.237005,0.230849,0.255339,0.257056,0.294750,0.243483,0.279549,0.255062,0.246356
5,0.315033,0.227815,0.220782,0.259671,0.237148,0.229105,0.282261,0.279559,0.260751,0.258025
6,0.314718,0.200722,0.229991,0.248088,0.249559,0.215012,0.236000,0.288277,0.248719,0.232437
7,0.313929,0.206077,0.190671,0.267184,0.215303,0.230439,0.249906,0.272492,0.260794,0.249879
8,0.313459,0.247087,0.243227,0.249336,0.259234,0.291806,0.247185,0.295535,0.248350,0.241601
9,0.311574,0.189773,0.216797,0.284533,0.244368,0.234659,0.250153,0.283825,0.248316,0.242411


...

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

['peeler',
 'thermometer',
 'nail clippers',
 'coaster',
 'glue container',
 'floss container',
 'chess piece',
 'box',
 'closed document folder',
 'envelope']

,white board eraser,sponge,butter,pencil,ice,paper towel,thermometer,floss container,shampoo bottle,marker,speaker,plastic cup,closed document folder,bottle cap,first aid kit,ruler,flashlight,peeler,nail clippers,coaster,glue container,chess piece,box,envelope
0,0.303464,0.295002,0.258483,0.236833,0.239518,0.280375,0.216864,0.282263,0.260386,0.211776,0.261135,0.221576,0.233587,0.238393,0.232718,0.238103,0.222307,0.233496,0.233232,0.260161,0.274269,0.234858,0.246970,0.231184
1,0.293410,0.222489,0.255989,0.255745,0.242810,0.258343,0.245927,0.299881,0.261760,0.246333,0.229645,0.241899,0.246751,0.240562,0.262828,0.254693,0.259942,0.239465,0.241227,0.221168,0.300784,0.230397,0.255033,0.245443
2,0.298491,0.219383,0.201150,0.293548,0.246040,0.239560,0.261804,0.251967,0.225007,0.249951,0.222113,0.243270,0.263548,0.231310,0.203718,0.274296,0.245356,0.242813,0.251431,0.221085,0.263548,0.219882,0.252068,0.246079
3,0.288841,0.242256,0.246455,0.236184,0.290110,0.288651,0.224710,0.292181,0.264345,0.219509,0.221696,0.273787,0.251979,0.254799,0.243193,0.249307,0.242005,0.240213,0.229196,0.254232,0.285004,0.242392,0.260203,0.253857
4,0.290393,0.242827,0.237868,0.245216,0.259751,0.327010,0.234212,0.278376,0.258547,0.237591,0.232999,0.256602,0.255677,0.237775,0.230991,0.254485,0.225120,0.237139,0.218793,0.250416,0.278098,0.226510,0.248888,0.286894
5,0.289771,0.227005,0.225529,0.256798,0.239823,0.242175,0.298820,0.281231,0.266168,0.255906,0.226669,0.245880,0.242907,0.234581,0.239525,0.265541,0.272607,0.260949,0.273532,0.207578,0.263331,0.246595,0.230794,0.230130
6,0.292945,0.199357,0.238914,0.236956,0.250392,0.227357,0.232204,0.316026,0.253193,0.227154,0.233047,0.230840,0.233304,0.265379,0.242580,0.217383,0.230794,0.222822,0.242731,0.235340,0.279661,0.218221,0.243534,0.221563
7,0.288801,0.207042,0.196862,0.264541,0.217536,0.249033,0.250697,0.274994,0.279739,0.246924,0.236133,0.258678,0.240859,0.261036,0.216241,0.232119,0.257482,0.237036,0.240264,0.210885,0.271185,0.223543,0.235184,0.224658
8,0.284606,0.252527,0.250214,0.239046,0.264014,0.326780,0.240296,0.290883,0.252185,0.231433,0.222552,0.249746,0.243803,0.234920,0.259903,0.245361,0.227915,0.236737,0.233077,0.258230,0.288892,0.239922,0.246086,0.273211
9,0.288580,0.188864,0.222467,0.281364,0.244494,0.249904,0.249208,0.288986,0.253383,0.252367,0.203063,0.245318,0.233295,0.256868,0.243943,0.242775,0.257142,0.247193,0.260465,0.212279,0.306919,0.240702,0.228025,0.226131


,white board eraser,sponge,butter,pencil,ice,paper towel,thermometer,floss container,shampoo bottle,marker,speaker,plastic cup,closed document folder,bottle cap,first aid kit,ruler,flashlight,peeler,nail clippers,coaster,glue container,chess piece,box,envelope
0,0.292664,0.303170,0.263162,0.230857,0.238970,0.287238,0.213826,0.288870,0.260696,0.205863,0.260501,0.221820,0.231884,0.236152,0.231822,0.234687,0.222196,0.233202,0.232903,0.260206,0.274049,0.232309,0.247004,0.229926
1,0.282483,0.222111,0.263689,0.250682,0.242302,0.263377,0.244588,0.309068,0.262762,0.241754,0.228314,0.242362,0.245549,0.238827,0.262906,0.252848,0.260891,0.239677,0.241570,0.220474,0.302049,0.228678,0.255162,0.243476
2,0.289821,0.217330,0.203269,0.298939,0.247253,0.243572,0.263383,0.256523,0.226505,0.248828,0.220759,0.244329,0.263847,0.230685,0.201418,0.274784,0.245518,0.244040,0.252957,0.219703,0.264122,0.219201,0.252840,0.245570
3,0.276736,0.242172,0.250796,0.232076,0.293271,0.297367,0.222737,0.299942,0.265488,0.213441,0.219596,0.275407,0.250824,0.252592,0.242084,0.246805,0.241688,0.240164,0.229182,0.253581,0.285814,0.240691,0.260486,0.252030
4,0.280229,0.243653,0.241999,0.240450,0.260016,0.338054,0.230696,0.284785,0.259418,0.232600,0.231412,0.257709,0.255909,0.235860,0.229766,0.252022,0.224411,0.237035,0.218817,0.250321,0.278401,0.224605,0.248935,0.286730
5,0.279691,0.225314,0.228331,0.254713,0.239997,0.245396,0.306038,0.287806,0.267772,0.253468,0.226370,0.245912,0.242017,0.233353,0.238917,0.265209,0.273108,0.261456,0.275506,0.206116,0.263951,0.245699,0.230546,0.228007
6,0.284367,0.197264,0.243300,0.231370,0.249546,0.229872,0.230570,0.333377,0.254281,0.223175,0.232415,0.232943,0.233201,0.265200,0.245024,0.213938,0.232037,0.222463,0.244422,0.235684,0.283403,0.215444,0.244604,0.218769
7,0.278818,0.205984,0.200665,0.262681,0.217658,0.254610,0.251880,0.282968,0.287038,0.244204,0.235351,0.261612,0.239873,0.262331,0.216068,0.230748,0.258839,0.237228,0.242368,0.210446,0.273289,0.223974,0.235719,0.223165
8,0.273395,0.253349,0.254173,0.234162,0.265176,0.338994,0.237819,0.295976,0.253373,0.225814,0.220745,0.250861,0.242766,0.232334,0.258864,0.242317,0.226915,0.236173,0.232623,0.257699,0.288937,0.238033,0.245821,0.272514
9,0.279853,0.187028,0.225955,0.279384,0.243748,0.253889,0.249334,0.299099,0.255161,0.254597,0.201138,0.246544,0.233172,0.256898,0.243920,0.240559,0.257632,0.247627,0.261299,0.211695,0.309054,0.238813,0.228849,0.225144


KeyboardInterrupt: 

In [147]:
firsts = {}
batch_size = 10
for x in tqdm(curr_firsts.itertuples()):
    closure(x.category, max_n=3*x.nfirst_x, firsts=firsts, show_display=True, batch_size=batch_size)
    print(firsts[x.category], x.nfirst_x)
    if x.nfirst_x <= batch_size:
        break

0it [00:00, ?it/s]

66 84.0
33 62.0
37 56.0
41 50.0
40 43.0
109 42.0
35 38.0
43 36.0
23 31.0
32 26.0
22 25.0
20 21.0
19 19.0
30 18.0
11 14.0
14 14.0
14 14.0
13 13.0
13 13.0
11 12.0
16 11.0
10 11.0
13 11.0
12 11.0
10 10.0


In [157]:
rdf = pd.concat([pd.Series(firsts).rename('feedback'), cf[['category', 'nfirst_x']].set_index('category')['nfirst_x'].rename('no_feedback')], axis=1)

In [158]:
((rdf.feedback < rdf.no_feedback).mean(), 
(rdf.feedback == rdf.no_feedback).mean(), 
 (rdf.feedback > rdf.no_feedback).mean())

(0.48, 0.2, 0.28)

In [171]:
rdf

,feedback,no_feedback
eraser (white board),66,84.0
placemat,33,62.0
hairclip,37,56.0
bookend,41,50.0
extension cable,40,43.0
walker,109,42.0
can opener,35,38.0
removable blade,43,36.0
squeeze bottle,23,31.0
nail (fastener),32,26.0


In [183]:
rdf.to_parquet('./data/objectnet_nfirst_verbal.parquet')